In [1]:
import cv2
import torch
import numpy as np
from time import time

# Cargar el modelo YOLOv5
def cargar_modelo():
    # Cargar YOLOv5 desde PyTorch Hub
    modelo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    # Configurar el modelo para inferencia
    modelo.conf = 0.25  # umbral de confianza
    modelo.iou = 0.45   # umbral de IoU
    return modelo

def procesar_video():
    # Inicializar la webcam (0 es generalmente la cámara integrada)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se pudo abrir la webcam.")
        return
    
    # Configurar dimensiones
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    # Cargar el modelo
    modelo = cargar_modelo()
    
    # Diccionario para contar objetos detectados
    contador = {}
    
    print("Presiona 'q' para salir")
    
    while True:
        # Capturar frame por frame
        ret, frame = cap.read()
        if not ret:
            print("Error: No se pudo leer el frame.")
            break
        
        tiempo_inicio = time()
        
        # Realizar detección
        resultados = modelo(frame)
        
        # Obtener información de los resultados
        detecciones = resultados.pandas().xyxy[0]  # detecciones en formato pandas
        
        # Mostrar los resultados en el frame
        frame_anotado = frame.copy()
        
        # Reiniciar contador para el frame actual
        contador_actual = {}
        
        # Procesar cada detección
        for i, deteccion in detecciones.iterrows():
            x1, y1, x2, y2 = int(deteccion['xmin']), int(deteccion['ymin']), int(deteccion['xmax']), int(deteccion['ymax'])
            clase = deteccion['name']
            confianza = deteccion['confidence']
            
            # Incrementar contador
            if clase in contador_actual:
                contador_actual[clase] += 1
            else:
                contador_actual[clase] = 1
            
            # Dibujar el rectángulo
            cv2.rectangle(frame_anotado, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Añadir etiqueta
            label = f"{clase}: {confianza:.2f}"
            cv2.putText(frame_anotado, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Actualizar contador global
        contador = contador_actual
        
        # Mostrar contador en la esquina
        y_pos = 30
        for clase, cantidad in contador.items():
            texto = f"{clase}: {cantidad}"
            cv2.putText(frame_anotado, texto, (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            y_pos += 30
        
        # Calcular FPS
        fps = 1.0 / (time() - tiempo_inicio)
        cv2.putText(frame_anotado, f"FPS: {fps:.1f}", (frame_anotado.shape[1] - 120, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Mostrar el frame resultante
        cv2.imshow('Detector de Objetos', frame_anotado)
        
        # Salir si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Liberar la captura y cerrar ventanas
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    procesar_video()

c:\Users\analyst4\AppData\Local\anaconda3\envs\counting\Lib\site-packages\torch\hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\analyst4/.cache\torch\hub\master.zip


ModuleNotFoundError: No module named 'pandas'